### Como utilizar ML com Dados Não Estruturais Textuais?

* Técnicas de NLP preprocessam os dados para filtrarmos o que é mais relevante para a classificação 

![Title](../imgs/nlp-illustration.png)


* A tokenização nos auxilia a dividir strings em palavras
* Essas palavras serão as "features" que serão utilizadas para aprendizado de um modelo de Machine Learning
* Um algoritmo de Machine Learning só entende números (atributos sempre numéricos), sendo assim, após a tokenização, precisamos converter nossas palavras para valores numéricos
* O processo de transformação textual para base numérico denomina-se **bag of words**

![Title](../imgs/1-Bag-of-words.png)

* Técnicas como TF-IDF e CountVectorizer, apesar de eficientes, transformam o dataset em dado tabular
  * Cada coluna é uma palavra
  * Quando a palavra existir em uma dada amostra, substituimos o valor 0 pela frequência em que aquela palavra ocorre no texto
  
  <img src="https://www.researchgate.net/profile/Haider-Al-Khateeb/publication/291950178/figure/fig1/AS:330186932408324@1455734107458/Term-Frequency-Inverse-Document-Frequency-TF-IDF.png" width=800>
  
* A consequencia disso é a perda da relação semântica nas frases
* Perda de contexto das sentenças
* Vocabulário imenso (maldição da dimensionalidade)

### Word Embeddings to rescue!

* Vetor multidimensional de palavras
* Mapa de características com a frequência de palavras e considerações semânticas 
* Baseado em similaridade
* Aprendizado por meio de redes neurais

##### Como fazemos isso?
* Dada uma palavra da frase
* Pegar as palavras vizinhas 
* Rede Neural irá calcular a probabilidade dessa palavra vizinha ser a palavra a ser predita
* Selecionamos as palavras vizinhas definindo uma "janela" ou "kernel"
  * Janela = 2 considere duas palavras ao redor da palavra atual
![Title](../imgs/training_data.png)

* Criamos um corpus com as palavras organizadas de acordo com as janelas em torno do dataset.
* Palavras que aparecerem juntas tem maior probabilidade de terem significado semântico
  * Exemplo: "soviética" provavelmente terá uma semelhança com união
  
 ![Title](../imgs/vetor_multi.png)
 
[Leitura adicional](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/): funcionamento de uma rede neural para embeddings

### Arquiteturas Word Embeddings

* Existem diferentes formas de gerar nossas embeddings, gerando determinadas arquiteturas previamente conhecidos como CBOW e SkipGram. Essas arquiteturas também são denominadas como tipo word2vec (palavra para vetor)

* CBOW: predizer uma palavra baseado nas palavras vizinhas. Mais rápido, funciona melhor nas palavras mais frequentes.
* Skip-Gram: predizer o contexto baseado nas palavras vizinhas. Funciona melhor com datasets menores
<img src="https://leimao.github.io/images/article/2019-08-23-Word2Vec-Classic/word2vec.png">
* Ambas arquiteturas tem uma ambientação semelhante, modificando a forma como a saída e a entrada são organizadas
* Exemplo: Hoje vai fazer sol pela manhã com pancadas de chuva à tarde
* Utilizando CBOW\
![Title](../imgs/cbow.png)
* Utilizando Skip-Gram\
![Title](../imgs/skip_gram.png)

[Leitura adicional](https://www.tensorflow.org/tutorials/text/word2vec): transformando o corpus do CBOW/skip_gram em numérico

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec

# Baixa as listas de stopwords e as tokenizações
nltk.download('stopwords')
nltk.download('punkt')

# Define as stopwords em inglês
sw_english = set(stopwords.words('english'))

# Instância o PorterStemmer
stemmer = PorterStemmer()

# Carrega o conjunto de dados
movies = pd.read_csv('../files/movies.csv', index_col = 0)

# Retira uma amostra do conjunto de dados
movies_sample = movies.sample(frac = 0.1, replace=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/Faaeel06/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/Faaeel06/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
movies_sample

,text,label
14099,Winchester '73 was the film that moved Mann fr...,1
4128,"""Citizen X"" tells the story of ""The Butcher of...",1
718,"After three hours in the Cinema hall,the stron...",0
3343,Really bad. Why anyone thinks this is a good f...,0
229,Quite a lot has been said about this film and ...,1
...,...,...
9310,"Bah. Another tired, desultory reworking of an ...",0
9486,Alexander Nevsky (1938) is a brilliant piece o...,1
24944,"When I first popped in Happy Birthday to Me, I...",0
36605,Well I watched this last night and the one thi...,0


In [9]:
# Cria um pipeline
def preprocessing(string):
    ###
    # Deixa apenas elementos alfanuméricos
    string = re.sub(r"[^a-zA-Z0-9]+", ' ', string)
    ###
    # deixa todas as palavras minúsculas
    string = string.lower()
    ###
    # tokenização
    words = word_tokenize(string)
    ###
    # Remove Stopwords
    filtered_words = []
    for w in words:
        if w not in sw_english:
            filtered_words.append(w)
    ###
    # Aplica o Stemming
    stem_words = []
    for w in filtered_words:
        s_words = stemmer.stem(w)
        stem_words.append(s_words)
    ###
    # Retorna a lista de palavras pré-processadas
    return stem_words

# Aplica o preprocessing nas críticas de filmes
movies_sample["filtered_words"] = movies_sample['text'].apply(lambda x: preprocessing(x))

In [10]:
movies_sample

,text,label,filtered_words
14099,Winchester '73 was the film that moved Mann fr...,1,"[winchest, 73, film, move, mann, b, movi, big,..."
4128,"""Citizen X"" tells the story of ""The Butcher of...",1,"[citizen, x, tell, stori, butcher, rostov, nic..."
718,"After three hours in the Cinema hall,the stron...",0,"[three, hour, cinema, hall, strongest, impress..."
3343,Really bad. Why anyone thinks this is a good f...,0,"[realli, bad, anyon, think, good, film, let, a..."
229,Quite a lot has been said about this film and ...,1,"[quit, lot, said, film, landmark, import, form..."
...,...,...,...
9310,"Bah. Another tired, desultory reworking of an ...",0,"[bah, anoth, tire, desultori, rework, copyrigh..."
9486,Alexander Nevsky (1938) is a brilliant piece o...,1,"[alexand, nevski, 1938, brilliant, piec, cinem..."
24944,"When I first popped in Happy Birthday to Me, I...",0,"[first, pop, happi, birthday, check, timer, se..."
36605,Well I watched this last night and the one thi...,0,"[well, watch, last, night, one, thing, make, c..."


In [ ]:
model = Word2Vec(movies_sample['filtered_words'])
model = Word2Vec(
        sentences = movies_sample['Filtered_words'],
        vector_size= 100,
        min_count= 1,
        workers=8,
        window=5,
        sg=1, #### 1 para skip-gram, 0 para CBOW
)

In [22]:
words = list(model.wv.index_to_key)

In [27]:
print("Primeiras 10 palavras:", words[:10], "\n\n")
print("Palavras similares a time: \n", model.wv.most_similar('time'), "\n\n")
print('Vetor para time:', model.wv.get_vector('time', norm=True), '\n\n')

Primeiras 10 palavras: ['br', 'movi', 'film', 'one', 'like', 'time', 'good', 'make', 'get', 'charact'] 


Palavras similares a time: 
 [('long', 0.8105802536010742), ('day', 0.7777706980705261), ('hour', 0.7667600512504578), ('rewatch', 0.7655228972434998), ('minut', 0.7577606439590454), ('spent', 0.7444809675216675), ('rememb', 0.7436185479164124), ('episod', 0.7387689352035522), ('spend', 0.7383964657783508), ('money', 0.7338274121284485)] 


Vetor para time: [-0.079018    0.23461457 -0.00283061 -0.09199482 -0.02540968 -0.02142855
  0.04741906  0.16059865 -0.16620846 -0.12922662 -0.04197725 -0.01233783
 -0.10628784 -0.13860823  0.02164914 -0.08192834  0.06432919 -0.11845332
 -0.16049783 -0.11077248 -0.04205097  0.05053041  0.22148465 -0.1384986
 -0.1388212  -0.00153927 -0.06091898 -0.07174087 -0.0271758   0.01391348
  0.03823535 -0.18785286 -0.01359395 -0.16428015 -0.04528658  0.16771941
  0.05274598  0.08715541 -0.04662609  0.03533069  0.07265826 -0.09664395
 -0.00660456  0.05697339

In [29]:
x = model.wv[words]
x

array([[ 0.9189932 , -0.30477402, -0.49083182, ..., -0.55766755,
        -0.55950606,  0.36208224],
       [-0.6183709 ,  0.5948241 ,  0.09102438, ..., -0.5801771 ,
        -0.59225184, -0.6436222 ],
       [-0.40518454,  0.6943868 , -0.04734575, ..., -1.0547723 ,
        -0.42749998, -0.48876402],
       ...,
       [-0.04310771,  0.07033185, -0.00332596, ..., -0.04638806,
         0.00865687,  0.03971612],
       [-0.0370767 ,  0.06001392, -0.00220027, ..., -0.05178751,
         0.01500952,  0.02939758],
       [-0.02087016,  0.0354037 ,  0.01262193, ..., -0.02827134,
         0.01310173,  0.01802594]], dtype=float32)

In [34]:
model.wv.similarity('time','second')

0.7314542

### Treinamento de ML

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(movies_sample["filtered_words"],
                                                    movies_sample['label'],
                                                    test_size=0.3,
                                                    random_state=42)

In [36]:
model = Word2Vec(X_train)

In [38]:
words = list(model.wv.index_to_key)
X_train_vect = np.array([np.array([model.wv[i] for i in ls if i in words]) for ls in X_train])
X_test_vect = np.array([np.array([model.wv[i] for i in ls if i in words]) for ls in X_test])

/tmp/ipykernel_12174/4006831418.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([model.wv[i] for i in ls if i in words]) for ls in X_train])
/tmp/ipykernel_12174/4006831418.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([model.wv[i] for i in ls if i in words]) for ls in X_test])


In [39]:
X_test_vect

array([array([[-0.22289124,  0.32768807, -0.09152499, ..., -0.46966347,
               -0.07278646, -0.3690154 ],
              [-0.21009372,  0.25849313,  0.13271204, ..., -0.34385616,
                0.2316736 , -0.04882853],
              [-0.11629928,  0.13996899,  0.07906803, ..., -0.1912492 ,
                0.13345686, -0.02674216],
              ...,
              [-0.29447538,  0.19703929, -0.00130964, ..., -0.34480038,
                0.11303307, -0.22679123],
              [-0.04687925,  0.05918838,  0.03206633, ..., -0.0890332 ,
                0.05313536, -0.00996448],
              [-0.22310989,  0.08101626,  0.03454849, ..., -0.23812021,
                0.00893255, -0.06481752]], dtype=float32)              ,
       array([[-0.22794406, -0.21178688, -0.19366407, ..., -0.10597081,
               -0.17505921, -0.00857241],
              [-0.26960555, -0.24301074, -0.02814163, ..., -0.0238598 ,
               -0.10245433, -0.02428582],
              [-0.22289124,  0.3276880

In [ ]:
model = RandomForestClassifier()
model.fit(X_train_vect, y_train)
y_pred = model.predict(X_test_vect)

print(classification_report(y_test, y_pred))